### class

In [ ]:
class FlagSchema(object):
    def __init__(self, flag_map: dict[str, int]):
        self.flag_map = flag_map
        self._bitfield_length = max(1, sum(flag_map.values()).bit_length())

    def decode(self, bitfield: int) -> list[str]:
        return [name for name, value in self.flag_map.items() if bitfield & value]

    def apply_to_df(self, df: pd.DataFrame) -> pd.Series:
        flag_srs = pd.Series(index=df.index, dtype=pd.Int64Dtype()).fillna(0)
        for k, v in self.flag_map.items():
            flag_srs += df[k].astype(bool) * v
        return flag_srs

### usages

In [ ]:
flag_map = {
    "has_survived": 0b0001,
    "is_alone": 0b0010,
    "is_male": 0b0100,
    "age_below_18": 0b1000,
}
fs = FlagSchema(flag_map=flag_map)
fs

In [ ]:
df["has_survived"] = df["survived"].astype(bool)
df["is_alone"] = df["alone"].astype(bool)
df["is_male"] = df["sex"].map({"male": 1, "female": 0})
df["age_below_18"] = df["age"] < 18

df["flags"] = fs.apply_to_df(df=df)
df["flags"]

0      4
1      1
2      3
3      1
4      6
      ..
886    6
887    3
888    0
889    7
890    6
Name: flags, Length: 891, dtype: Int64

In [ ]:
(df["flags"] & (fs.flag_map["has_survived"] | fs.flag_map["is_alone"])) == (
    fs.flag_map["has_survived"] | fs.flag_map["is_alone"]
)

0      False
1      False
2       True
3      False
4      False
       ...  
886    False
887     True
888    False
889     True
890    False
Name: flags, Length: 891, dtype: boolean

In [ ]:
df["flags"].apply(lambda x: fs.decode(x))

0                              [is_male]
1                         [has_survived]
2               [has_survived, is_alone]
3                         [has_survived]
4                    [is_alone, is_male]
                     ...                
886                  [is_alone, is_male]
887             [has_survived, is_alone]
888                                   []
889    [has_survived, is_alone, is_male]
890                  [is_alone, is_male]
Name: flags, Length: 891, dtype: object